# 5. Hyperparameter Tuning

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn

In [2]:
#Read in the dataframes
feat_df = pd.read_pickle('../data/features_df.pkl')
meta_df = pd.read_csv('../data/speechdetails.csv')

#Removes all the text that was in before
feat_df = feat_df.select_dtypes(exclude=['object'])

#Isolate predictive (X) and target (y) variables
X = feat_df.values
y = meta_df['IC'].values

#Implement train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, 
                                                    test_size=0.2,
                                                    random_state=11)

## XGBoost

In [3]:
params = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "alpha"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
 "grow_policy"      : ['depthwise','lossguide'] }

In [4]:
grid_search_dict =  {"learning_rate"    : list() ,
 "max_depth"        : list(),
 "min_child_weight" : list(),
 "gamma"            : list(),
 "colsample_bytree" : list(),
 "rmse" : list()}

In [8]:
for lr in params['learning_rate']:
    for md in params['max_depth']:
        for cw in params['min_child_weight']:
            for g in params['gamma']:
                for cbt in params['colsample_bytree']:
                    grid_search_dict['learning_rate'].append(lr)
                    grid_search_dict['max_depth'].append(md)
                    grid_search_dict['min_child_weight'].append(cw)
                    grid_search_dict['gamma'].append(g)
                    grid_search_dict['colsample_bytree'].append(cbt)
                    
                    xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                    max_depth = 5, alpha = 10, n_estimators = 30)

                    xg_reg.fit(X_train, y_train)

                    preds = xg_reg.predict(X_test)
                    grid_search_dict['rmse'].append(np.sqrt(metrics.mean_squared_error(y_test, preds)))


In [11]:
df = pd.DataFrame().from_dict(grid_search_dict)

In [16]:
df = pd.read_pickle('../data/hp_tune_xgboost.pkl')

In [20]:
len(df)

38400

In [ ]:
optimal_params = {"learning_rate" : 0.1,
 "max_depth"        : 3,
 "min_child_weight" : 1,
 "gamma"            : 0,
 "alpha"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : 0.3,
 "grow_policy"      : ['depthwise','lossguide']}

In [22]:
df.sort_values('rmse',ascending=True).head()

,learning_rate,max_depth,min_child_weight,gamma,alpha,colsample_bytree,grow_policy,rmse
6416,0.1,3,1,0.0,0.2,0.3,depthwise,0.327255
6424,0.1,3,1,0.0,0.3,0.3,depthwise,0.327255
6425,0.1,3,1,0.0,0.3,0.3,lossguide,0.327255
6409,0.1,3,1,0.0,0.1,0.3,lossguide,0.327255
6432,0.1,3,1,0.0,0.4,0.3,depthwise,0.327255


## Sequential Model

In [23]:
df = pd.read_pickle('../data/hp_tune_sequential.pkl')

In [24]:
len(df)

216

In [25]:
df.sort_values('rmse',ascending=True).head()

,layer1,layer2,layer3,layer4,epochs,optimizer,losses,rmse
149,4,16,8,8,150,RMSprop,mse,0.336585
151,4,16,8,4,150,adam,mse,0.336976
143,8,4,4,4,150,RMSprop,mse,0.337028
203,4,4,8,4,150,RMSprop,mse,0.337248
53,16,4,8,8,150,RMSprop,mse,0.338083


In [ ]:
optimal_params = {"layer1" : 4,
 "layer2" : 16,
 "layer3" : 8,
 "layer4" : 8,
 "epochs" : 150,
 "optimizer" : 'RMSprop',
 "losses"      : 'mse'}

In [ ]:
params = {
     'layer1' : [16, 8, 4],
     'layer2' : [16, 8, 4],
     'layer3' : [8, 4],
     'layer4' : [8, 4],
     'epochs': [150],
     'optimizer': ['adam', 'adagrad','RMSprop'],
     'losses': ['binary_crossentropy','mse']}